In [69]:
import pandas as pd
import tkinter
from tkinter import *

In [70]:
class POS:
    def __init__(self):
        self.inventory = {}
        self.cart = {}

    def read_inventory(self):
        inventory = pd.read_csv('Inventory.csv')

    def add_to_cart(self, name, quantity):
        if name in self.inventory:
            if name in self.cart:
                self.cart[name] += quantity
            else:
                self.cart[name] = quantity
        else:
            print("Item not found or insufficient quantity.")

    def calculate_total(self, payment_method):
        total = 0
        for item, quantity in self.cart.items():
            total += self.inventory[item]['price'] * quantity
        if payment_method == 'credit':
            total *= .025
        return total

    def checkout(self, payment_method):
        total = self.calculate_total(payment_method)
        print("Total: $", total)
        self.cart = {}

pos = POS()

In [71]:
inventory = pd.read_csv('Inventory.csv')
inventory['Price'] = inventory['Price'].str.replace('$', '').str.replace(',', '').astype(float)
inventory = inventory.sort_values(['Category', 'Item', 'Package'])
inventory.head()

,Item,Category,Package,Price
1,Cinnamon Milk Bread,Bread,Single,5.5
0,Sesame Milk Bread,Bread,Single,5.5
5,Double Chocolate Cookie,Cookie,Box,21.0
4,Double Chocolate Cookie,Cookie,Single,5.5
3,Toffee Cookie,Cookie,Box,22.0


In [72]:
price_dict = inventory.drop('Category', axis=1).set_index(['Item', 'Package']).to_dict()['Price']
price_dict

{('Cinnamon Milk Bread', 'Single'): 5.5,
 ('Sesame Milk Bread', 'Single'): 5.5,
 ('Double Chocolate Cookie', 'Box'): 21.0,
 ('Double Chocolate Cookie', 'Single'): 5.5,
 ('Toffee Cookie', 'Box'): 22.0,
 ('Toffee Cookie', 'Single'): 5.75}

In [73]:
packaging = inventory[['Item', 'Package']].groupby('Item').agg(list).to_dict()['Package']
packaging

{'Cinnamon Milk Bread': ['Single'],
 'Double Chocolate Cookie': ['Box', 'Single'],
 'Sesame Milk Bread': ['Single'],
 'Toffee Cookie': ['Box', 'Single']}

In [102]:
d = dict()
def inventory_dict_translate(row):
    item = row['Item']
    package = row['Package']
    price = row['Price']
    if item in d.keys():
        d[item][package] = price
    else:
        d[item] = {package:price}
    
inventory.apply(lambda x: dict_translate(x), axis=1)

d

{'Cinnamon Milk Bread': {'Single': 5.5},
 'Sesame Milk Bread': {'Single': 5.5},
 'Double Chocolate Cookie': {'Box': 21.0, 'Single': 5.5},
 'Toffee Cookie': {'Box': 22.0, 'Single': 5.75}}

In [189]:
import csv
inventory = dict()
with open('Inventory.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        item = row[0]
        package = row[2]
        price = row[3]

        if item in inventory.keys():
            inventory[item][package] = price
        else:
            inventory[item] = {package:price}
inventory

{'Item': {'Package': 'Price'},
 'Sesame Milk Bread': {'Single': '$5.50 '},
 'Cinnamon Milk Bread': {'Single': '$5.50 '},
 'Toffee Cookie': {'Single': '$5.75 ', 'Box': '$22.00 '},
 'Double Chocolate Cookie': {'Single': '$5.50 ', 'Box': '$21.00 '},
 'Smores Brownie': {'Single': '$8.00 ', 'Box': '$33.00 '}}

In [187]:
inventory

{'Smores Brownie': {'Box': '$33.00 '}}

In [234]:
import tkinter as tk
from tkinter import messagebox
from tkinter import font
from datetime import datetime
from pytz import timezone
import math
import csv

class POS:
    def __init__(self, master):
        self.master = master
        self.read_inventory()
        self.app_config()

    def read_inventory(self):
        inventory = dict()

        with open('Inventory.csv', 'r') as csv_file:
            
            csv_reader = csv.reader(csv_file, delimiter=',')

            for row in csv_reader:

                item = row[0]
                if item != 'Item':
                    package = row[2]
                    price = row[3].replace('$','').replace(' ','')
                    price = float(price)

                    if item in inventory.keys():
                        inventory[item][package] = price
                    else:
                        inventory[item] = {package:price}

        self.inventory = inventory

    def app_config(self):

        default_font = font.nametofont('TkDefaultFont')
        default_font.configure(size=18)
        self.master.title('Evjoy POS system')

        frm_item = tk.Frame(self.master, borderwidth=3, padx=5, pady=5)
        frm_item.pack(fill='x')

        max_cols = 0

        for i, item in enumerate(self.inventory.keys()):
            lbl_item = tk.Label(frm_item, text=item)
            lbl_item.grid(row=i, column=0, sticky='w')
            for j,package in enumerate(self.inventory[item].keys()):
                max_cols = j if j > max_cols else max_cols
                btn_product = tk.Button(frm_item, text=package, command=lambda x=(item,package):self.add_item(f"{x[0]}-{x[1]}", self.inventory[x[0]][x[1]]))
                btn_product.grid(row=i, column=j+1, sticky='ew')

        frm_item.columnconfigure(0, weight=1)
        for col in range(max_cols+1):
            frm_item.columnconfigure(col+1, weight=1)

        frm_cart = tk.Frame(self.master, borderwidth=3, padx=5, pady=5)
        frm_cart.pack(fill='x')
        frm_cart.columnconfigure([0,1], weight=1)
        lbl_cart = tk.Label(frm_cart, text="Cart:")
        lbl_cart.grid(row=0, column=0, sticky='w')
        self.cart_text = tk.Text(frm_cart, height=10, width=40)
        self.cart_text.grid(row=1, column=0, columnspan=2, sticky='ew')
        self.subtotal_label = tk.Label(frm_cart, text="Subtotal: $0.00")
        self.subtotal_label.grid(row=3, column=0, columnspan=2)

        self.cart = []
        self.subtotal = 0
        self.total = 0

        frm_final = tk.Frame(self.master, padx=5, pady=5)
        frm_final.pack()
        btn_cash = tk.Button(frm_final, text='Cash Payment', command=lambda: self.calc_total('cash'))
        btn_cash.grid(row=0, column=0)
        btn_credit = tk.Button(frm_final, text='Credit Payment', command=lambda: self.calc_total('credit'))
        btn_credit.grid(row=0, column=1)
        self.lbl_total = tk.Label(frm_final, text='Total: $0.00')
        self.lbl_total.grid(row=1,column=0, columnspan=2)
        btn_clear = tk.Button(frm_final, text='Cancel/Clear Form', command=lambda: self.cancel_order())
        btn_clear.grid(row=2, column=0)
        self.btn_submit = tk.Button(frm_final, text='Submit Order Record', command=lambda: self.submit_order(), state=tk.DISABLED)
        self.btn_submit.grid(row=2, column=1)
        

    def add_item(self, name, price):
        self.cart.append((name, price))
        self.update_cart()
        self.btn_submit.config(state=tk.DISABLED)

    def update_cart(self):
        self.cart_text.delete("1.0", tk.END)
        for item, price in self.cart:
            self.cart_text.insert(tk.END, f"{item} - ${price:.2f}\n")
        if self.cart:
            self.subtotal += price
        else:
            self.subtotal = 0.0
        self.subtotal_label.config(text=f"Subtotal: ${self.subtotal:.2f}")

    def calc_total(self, payment_method):
        if payment_method == 'cash':
            self.total = self.subtotal
        else:
            self.total = math.ceil((self.subtotal * 1.035 + .1) * 100) / 100
        self.lbl_total.config(text=f"Total: ${self.total:.2f}")
        self.btn_submit.config(state=tk.ACTIVE)

    def submit_order(self):
        self.record_order()
        messagebox.showinfo(message=f"Recorded order for ${self.total}")
        self.cart = []
        self.total = 0.0
        self.subtotal = 0.0
        self.update_cart()
        self.calc_total('cash')
        self.btn_submit.config(state=tk.DISABLED)

    def cancel_order(self):
        self.cart = []
        self.total = 0.0
        self.subtotal = 0.0
        self.update_cart()
        self.calc_total('cash')
        self.btn_submit.config(state=tk.DISABLED)

    def record_order(self):
        with open('Purchase_History.csv', 'a') as fd:
            for i in self.cart:
                item_package = i[0].split('-')
                tz = timezone('EST')
                timestamp = datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')
                fd.write(','.join([item_package[0], item_package[1], str(i[1]), timestamp])+'\n')
        
        



root = tk.Tk()
pos = POS(root)
root.mainloop()

In [201]:
from tkinter import font

font.nametofont('TkDefaultFont')

RuntimeError: Too early to use font: no default root window

In [146]:
from datetime import datetime
from pytz import timezone

items = []
packages = []
prices = []
timestamp = []
for i in order:
    item_package = i[0].split('-')
    items.append(item_package[0])
    packages.append(item_package[1])
    prices.append(i[1])
    tz = timezone('EST')
    timestamp.append(datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S'))
    
pd.DataFrame({'Item':items, 'Package':packages,'Price':prices, 'Timestamp':timestamp})

,Items,Packages,Prices,Timestamp
0,Cinnamon Milk Bread,Single,5.50,2024-12-17 16:33:16
1,Sesame Milk Bread,Single,5.50,2024-12-17 16:33:16
2,Double Chocolate Cookie,Box,21.00,2024-12-17 16:33:16
3,Toffee Cookie,Single,5.75,2024-12-17 16:33:16


In [ ]:
with open('Purchase_History.csv', 'a') as fd:
    for i in order:
        item_package = i[0].split('-')
        tz = timezone('EST')
        timestamp = datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')
        fd.write(','.join([item_package[0], item_package[1], str(i[1]), timestamp])+'\n')
    

In [148]:
pd.read_csv('Purchase_History.csv')

,Item,Package,Price,Timestamp


In [81]:
import tkinter as tk

class POS:
    def __init__(self, master):
        self.master = master
        master.title("Simple POS System")

        # Product buttons
        self.button1 = tk.Button(master, text="Product 1", command=lambda: self.add_item("Product 1", 10.0))
        self.button1.grid(row=0, column=0)
        self.button2 = tk.Button(master, text="Product 2", command=lambda: self.add_item("Product 2", 15.0))
        self.button2.grid(row=0, column=1)

        # Cart display
        self.cart_label = tk.Label(master, text="Cart:")
        self.cart_label.grid(row=1, column=0)
        self.cart_text = tk.Text(master, height=5, width=20)
        self.cart_text.grid(row=2, column=0, columnspan=2)

        # Total
        self.total_label = tk.Label(master, text="Total: $0.00")
        self.total_label.grid(row=3, column=0, columnspan=2)

        # Checkout button
        self.checkout_button = tk.Button(master, text="Checkout", command=self.checkout)
        self.checkout_button.grid(row=4, column=0, columnspan=2)

        self.cart = []
        self.total = 0.0

    def add_item(self, name, price):
        self.cart.append((name, price))
        self.update_cart()

    def update_cart(self):
        self.cart_text.delete("1.0", tk.END)
        for item, price in self.cart:
            self.cart_text.insert(tk.END, f"{item} - ${price:.2f}\n")
        self.total += price
        self.total_label.config(text=f"Total: ${self.total:.2f}")

    def submit_order(self):
        # Implement checkout logic here
        # tk.messagebox.showinfo("Checkout", "Thank you for your purchase!")
        self.cart = []
        self.total = 0.0
        self.update_cart()

    def cancel_order(self):
        self.cart = []
        self.total = 0.0
        self.update_cart()

root = tk.Tk()
pos = POS(root)
root.mainloop()


In [68]:
import tkinter as tk
import tkinter.ttk as ttk

window = tk.Tk()

labels = ['First Name', 'Last Name','Address Line 1', 'Address Line 2', 'City', 'State/Province', 'Postal Code', 'Contry']
frm_form = tk.Frame(relief=tk.SUNKEN, borderwidth=3)
frm_form.columnconfigure(1, weight=1, minsize=50)
frm_form.pack(fill=tk.X)

for i, label in enumerate(labels):
    lbl = tk.Label(master=frm_form, text=label+':')
    lbl.grid(row=i,column=0, sticky='ew')
    ent = tk.Entry(master=frm_form)
    ent.grid(row=i, column=1, sticky='ew')

frm_btns = tk.Frame()
frm_btns.pack(fill=tk.X)
btn_clr = tk.Button(master=frm_btns, text='Clear')
btn_clr.pack(side=tk.RIGHT, ipadx=10)

btn_sbmt = tk.Button(master=frm_btns, text='Submit')
btn_sbmt.pack(side=tk.RIGHT, padx=10, ipadx=10)


window.mainloop()
